# **Part 0**

https://archive.ics.uci.edu/ml/machine-learning-databases/00464/superconduct.zip

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
data = pd.read_csv("train.csv")

data = data.dropna()

quantiles = np.quantile(data["critical_temp"], np.arange(0, 1.1, 0.1))

e1 = data[data["critical_temp"] < 3.32]
e2 = data[(data["critical_temp"] > 3.32) & (data["critical_temp"] < 8.4)]
e3 = data[(data["critical_temp"] > 8.4) & (data["critical_temp"] < 20)]
e4 = data[(data["critical_temp"] > 20) & (data["critical_temp"] < 40.56667)]
e5 = data[(data["critical_temp"] > 40.56667) & (data["critical_temp"] < 79)]
e6 = data[data["critical_temp"] > 79]

column_names = pd.DataFrame(data.columns)

data.head()


In [ ]:
# Add class labels, drop temperature
e1["label"] = "CL1"
e1 = e1.drop("critical_temp", axis=1)

e2["label"] = "CL2"
e2 = e2.drop("critical_temp", axis=1)

e3["label"] = "CL3"
e3 = e3.drop("critical_temp", axis=1)

e4["label"] = "CL4"
e4 = e4.drop("critical_temp", axis=1)

e5["label"] = "CL5"
e5 = e5.drop("critical_temp", axis=1)

e6["label"] = "CL6"
e6 = e6.drop("critical_temp", axis=1)

e1.head()


# **Part 1**

In [ ]:
#Q1: We dont need to use SMOTE and one-hot encoding
# Randomly select 80% of cases from each class to create the training set
train_e1, test_e1 = train_test_split(e1, test_size=0.2, random_state=42)
train_e2, test_e2 = train_test_split(e2, test_size=0.2, random_state=42)
train_e3, test_e3 = train_test_split(e3, test_size=0.2, random_state=42)
train_e4, test_e4 = train_test_split(e4, test_size=0.2, random_state=42)
train_e5, test_e5 = train_test_split(e5, test_size=0.2, random_state=42)
train_e6, test_e6 = train_test_split(e6, test_size=0.2, random_state=42)

# Concatenate the training and test sets for each class to create the final TRAIN and TEST sets
train_data = pd.concat([train_e1, train_e2, train_e3, train_e4, train_e5, train_e6])
test_data = pd.concat([test_e1, test_e2, test_e3, test_e4, test_e5, test_e6])


In [ ]:
# One-hot encoding
# train data
one_hot_train = pd.get_dummies(train_data["label"])
train_data = train_data.drop("label", axis=1)
train_data = pd.concat([train_data, one_hot_train], axis=1)

# test data
one_hot_test = pd.get_dummies(test_data["label"])
test_data = test_data.drop("label", axis=1)
test_data = pd.concat([test_data, one_hot_test], axis=1)

train_data.head()


# **Part 2**

In [ ]:
#Q2:
# Split the dataset into features and target variable
en_col = ["CL1", "CL2", "CL3", "CL4", "CL5", "CL6"]

x_train = train_data.drop(en_col, axis=1)
y_train = train_data[en_col]
x_test = test_data.drop(en_col, axis=1)
y_test = test_data[en_col]

# Standardize the features
scaler = StandardScaler()
x_train = pd.DataFrame(scaler.fit_transform(x_train))
x_test = pd.DataFrame(scaler.transform(x_test))

x_train.head()


In [ ]:
from tensorflow.keras.initializers import RandomUniform
import time

# Validation Split
x_train1, x_valid, y_train1, y_valid = train_test_split(x_train, y_train, test_size=0.1)

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(41, activation='relu', input_shape=(x_train.shape[1],), kernel_initializer=RandomUniform),
    tf.keras.layers.Dense(y_train.shape[1], activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
start_time = time.time()

history = model.fit(x_train1, y_train1, batch_size=152, epochs=175, verbose=0, validation_data=(x_valid,y_valid))

end_time = time.time()

time_per_epoch = (end_time - start_time) / len(history.history['loss'])

# Calculate average cross-entropy loss for train and test sets
train_preds = model.predict(x_train)
train_avCRE = log_loss(y_train, train_preds) / len(x_train)

test_preds = model.predict(x_test)
test_avCRE = log_loss(y_test, test_preds) / len(x_test)

# Extract the training and validation losses for each epoch
train_losses_per_epoch = history.history['loss']
test_losses_per_epoch = history.history['val_loss']

# Calculate the average cross-entropy loss per epoch for train and test sets
train_avCRE_per_epoch = [loss / len(x_train) for loss in train_losses_per_epoch]
test_avCRE_per_epoch = [loss / len(x_test) for loss in test_losses_per_epoch]

print("Average Cross Entropy Loss for Train Set:", train_avCRE)
print("Average Cross Entropy Loss for Test Set:", test_avCRE)

print("Average Cross Entropy Loss per Epoch for Train Set:", train_avCRE_per_epoch)
print("Average Cross Entropy Loss per Epoch for Test Set:", test_avCRE_per_epoch)

print("Computing time per epoch: ", time_per_epoch)


In [ ]:
# 150 epochs was originally used, but the accuracy of the training set was still
# continuing to decrease, so I bumped it up to 200
# Computation time for 150 epochs: ~3.5 min
# Computation time for 200 epochs: ~4.5 min
# Computation time for 250 epochs: ~5.5 min
# We will use 150 epochs going forward. Test accuracy levels off at around this point.
# 200 and 250 epochs were computed to make sure no accuracy gains were left on the table.

import matplotlib.pyplot as plt

# Plot the loss per epoch
plt.plot(train_losses_per_epoch, label='Train')
plt.plot(test_losses_per_epoch, label='Test')
plt.title('Cross-Entropy Loss per Epoch, h=41')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)

print("Test Accuracy: {:.2f}%".format(test_accuracy * 100))

In [ ]:
round(x_train1.shape[0]/100, 0)

In [ ]:
# Create and print the confusion matrix.

from sklearn.metrics import confusion_matrix

test_pred_labels = np.argmax(test_preds, axis=1)
y_test_array = y_test.to_numpy()
y_test_labels = np.argmax(y_test_array, axis=1)

# Compute the confusion matrix
conf_matrix = confusion_matrix(test_pred_labels,y_test_labels)

# Normalize the confusion matrix
normalized_conf_matrix = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]

# Convert to percentage
percentage_conf_matrix = normalized_conf_matrix * 100

print("Confusion Matrix (Percentage):")
print("Confusion Matrix (Percentage):")
for row in percentage_conf_matrix:
    formatted_row = ["{:.2f}".format(x) for x in row]
    print(" ".join(formatted_row))


# **Part 3**

In [ ]:
from sklearn.decomposition import PCA

# Create a PCA object with 90% variance explained
pca = PCA(n_components=0.9)

# Fit the PCA model on the training data
pca.fit(x_train)

# Transform the training data into the new feature space
x_train_pca = pca.transform(x_train)
x_test_pca = pca.transform(x_test)

x_train_pca = pd.DataFrame(x_train_pca)
x_test_pca = pd.DataFrame(x_test_pca)

# Print the shape of the transformed data
print("Shape of transformed data:", x_train_pca.shape)


In [ ]:
# Create a PCA object
pca = PCA()

# Fit the PCA model on the training data
pca.fit(x_train)

# Compute the cumulative sum of explained variance
var_exp = np.cumsum(pca.explained_variance_ratio_)

# Create a bar plot of explained variance vs. number of components
plt.figure(figsize=(8, 6))
plt.bar(range(1, len(var_exp) + 1), var_exp, align='center', label='Individual explained variance')
plt.step(range(1, len(var_exp) + 1), var_exp, where='mid', label='Cumulative explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal components')
plt.legend(loc='best')
plt.tight_layout()
plt.show()


In [ ]:
# Validation Split
x_train2, x_valid, y_train2, y_valid = train_test_split(x_train, y_train, test_size=0.1)

# Define the model
model2 = tf.keras.Sequential([
    tf.keras.layers.Dense(12, activation='relu', input_shape=(x_train.shape[1],), kernel_initializer=RandomUniform),
    tf.keras.layers.Dense(y_train.shape[1], activation='softmax')
])

# Compile the model
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
start_time = time.time()

history2 = model2.fit(x_train2, y_train2, batch_size=152, epochs=225, verbose=0, validation_data=(x_valid,y_valid))

end_time = time.time()

time_per_epoch2 = (end_time - start_time) / len(history2.history['loss'])

# Calculate average cross-entropy loss for train and test sets
train_preds2 = model2.predict(x_train)
train_avCRE2 = log_loss(y_train, train_preds2) / len(x_train)

test_preds2 = model2.predict(x_test)
test_avCRE2 = log_loss(y_test, test_preds2) / len(x_test)

# Extract the training and validation losses for each epoch
train_losses_per_epoch2 = history2.history['loss']
test_losses_per_epoch2 = history2.history['val_loss']

# Calculate the average cross-entropy loss per epoch for train and test sets
train_avCRE_per_epoch2 = [loss / len(x_train) for loss in train_losses_per_epoch2]
test_avCRE_per_epoch2 = [loss / len(x_test) for loss in test_losses_per_epoch2]

print("Average Cross Entropy Loss for Train Set:", train_avCRE2)
print("Average Cross Entropy Loss for Test Set:", test_avCRE2)

print("Average Cross Entropy Loss per Epoch for Train Set:", train_avCRE_per_epoch2)
print("Average Cross Entropy Loss per Epoch for Test Set:", test_avCRE_per_epoch2)

print("Computing time per epoch: ", time_per_epoch2)


In [ ]:
# Plot the loss per epoch
plt.plot(train_losses_per_epoch2, label='Train')
plt.plot(test_losses_per_epoch2, label='Test')
plt.title('Cross-Entropy Loss per Epoch, h_low=12')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Evaluate the model on the test set
test_loss2, test_accuracy2  = model2.evaluate(x_test, y_test, verbose=0)

print("Test Accuracy: {:.2f}%".format(test_accuracy2 * 100))

# Need to work out why this model has over 100% accuracy. Will come back to it.

# There is a higher loss than the first model, but not by much.


In [ ]:
test_pred_labels2 = np.argmax(test_preds2, axis=1)

# Compute the confusion matrix
conf_matrix2 = confusion_matrix(test_pred_labels2,y_test_labels)

# Normalize the confusion matrix
normalized_conf_matrix2 = conf_matrix2.astype('float') / conf_matrix2.sum(axis=1)[:, np.newaxis]

# Convert to percentage
percentage_conf_matrix2 = normalized_conf_matrix2 * 100

print("Confusion Matrix (Percentage):")
print("Confusion Matrix (Percentage):")
for row in percentage_conf_matrix2:
    formatted_row = ["{:.2f}".format(x) for x in row]
    print(" ".join(formatted_row))


# **Part 4**

In [ ]:
from sklearn.decomposition import PCA

# Define the variance threshold
var_threshold = 0.9

# Loop through each dataframe and perform PCA on numeric columns
for i, df in enumerate([e1, e2, e3, e4, e5, e6]):
    numeric_df = df.select_dtypes(include=['float64', 'int64'])
    pca = PCA(n_components=var_threshold)
    pca.fit(numeric_df)
    n_components = pca.n_components_
    print(f"Number of features that capture 90% of the variance for e{i+1}: {n_components}")

# Our variance is highly concentrated to only a few features for every class.
# h1 + h2 ... = 13, which is the same dimension used for h_low.
# Going to use h=p where p is the total number of features for h_high


In [ ]:
# Define the variance threshold
var_threshold = 0.9

# Define the number of principal components to try
max_components = 10

# Loop through each dataframe and perform PCA on numeric columns
for i, df in enumerate([e1, e2, e3, e4, e5, e6]):
    numeric_df = df.select_dtypes(include=['float64', 'int64'])
    pca = PCA(n_components=max_components)
    pca.fit(numeric_df)
    var_ratio = pca.explained_variance_ratio_
    cum_var_ratio = np.cumsum(var_ratio)
    n_components = np.argmax(cum_var_ratio >= var_threshold) + 1
    plt.plot(np.arange(1, max_components+1), cum_var_ratio, label=f'e{i+1}')
    
plt.xlabel('Number of principal components')
plt.ylabel('Percentage of explained variance')
plt.axhline(y=var_threshold, color='black', linestyle='--', label=f'{var_threshold:.0%} threshold')
plt.legend()
plt.show()


In [ ]:
# Validation Split
x_train3, x_valid, y_train3, y_valid = train_test_split(x_train, y_train, test_size=0.1)

# Define the model
model3 = tf.keras.Sequential([
    tf.keras.layers.Dense(13, activation='relu', input_shape=(x_train.shape[1],), kernel_initializer=RandomUniform),
    tf.keras.layers.Dense(y_train.shape[1], activation='softmax')
])

# Compile the model
model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
start_time = time.time()

history3 = model3.fit(x_train3, y_train3, batch_size=152, epochs=160, verbose=0, validation_data=(x_valid,y_valid))

end_time = time.time()

time_per_epoch3 = (end_time - start_time) / len(history3.history['loss'])

# Calculate average cross-entropy loss for train and test sets
train_preds3 = model3.predict(x_train)
train_avCRE3 = log_loss(y_train, train_preds3) / len(x_train)

test_preds3 = model3.predict(x_test)
test_avCRE3 = log_loss(y_test, test_preds3) / len(x_test)

# Extract the training and validation losses for each epoch
train_losses_per_epoch3 = history3.history['loss']
test_losses_per_epoch3 = history3.history['val_loss']

# Calculate the average cross-entropy loss per epoch for train and test sets
train_avCRE_per_epoch3 = [loss / len(x_train) for loss in train_losses_per_epoch3]
test_avCRE_per_epoch3 = [loss / len(x_test) for loss in test_losses_per_epoch3]

print("Average Cross Entropy Loss for Train Set:", train_avCRE3)
print("Average Cross Entropy Loss for Test Set:", test_avCRE3)

print("Average Cross Entropy Loss per Epoch for Train Set:", train_avCRE_per_epoch3)
print("Average Cross Entropy Loss per Epoch for Test Set:", test_avCRE_per_epoch3)

print("Computing time per epoch: ", time_per_epoch3)


In [ ]:
# Plot the loss per epoch
plt.plot(train_losses_per_epoch3, label='Train')
plt.plot(test_losses_per_epoch3, label='Test')
plt.title('Cross-Entropy Loss per Epoch,  h_3=13')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Evaluate the model on the test set
test_loss3, test_accuracy3  = model3.evaluate(x_test, y_test, verbose=0)

print("Test Accuracy: {:.2f}%".format(test_accuracy3 * 100))


In [ ]:
test_pred_labels3 = np.argmax(test_preds3, axis=1)

# Compute the confusion matrix
conf_matrix3 = confusion_matrix(test_pred_labels3,y_test_labels)

# Normalize the confusion matrix
normalized_conf_matrix3 = conf_matrix3.astype('float') / conf_matrix3.sum(axis=1)[:, np.newaxis]

# Convert to percentage
percentage_conf_matrix3 = normalized_conf_matrix3 * 100

print("Confusion Matrix (Percentage):")
print("Confusion Matrix (Percentage):")
for row in percentage_conf_matrix3:
    formatted_row = ["{:.2f}".format(x) for x in row]
    print(" ".join(formatted_row))


In [ ]:
# Validation Split
x_train4, x_valid, y_train4, y_valid = train_test_split(x_train, y_train, test_size=0.1)

# Define the model
model4 = tf.keras.Sequential([
    tf.keras.layers.Dense(81, activation='relu', input_shape=(x_train.shape[1],), kernel_initializer=RandomUniform),
    tf.keras.layers.Dense(y_train.shape[1], activation='softmax')
])

# Compile the model
model4.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
start_time = time.time()

history4 = model4.fit(x_train4, y_train4, batch_size=152, epochs=160, verbose=0, validation_data=(x_valid,y_valid))

end_time = time.time()

time_per_epoch4 = (end_time - start_time) / len(history4.history['loss'])

# Calculate average cross-entropy loss for train and test sets
train_preds4 = model4.predict(x_train)
train_avCRE4 = log_loss(y_train, train_preds4) / len(x_train)

test_preds4 = model4.predict(x_test)
test_avCRE4 = log_loss(y_test, test_preds4) / len(x_test)

# Extract the training and validation losses for each epoch
train_losses_per_epoch4 = history4.history['loss']
test_losses_per_epoch4 = history4.history['val_loss']

# Calculate the average cross-entropy loss per epoch for train and test sets
train_avCRE_per_epoch4 = [loss / len(x_train) for loss in train_losses_per_epoch4]
test_avCRE_per_epoch4 = [loss / len(x_test) for loss in test_losses_per_epoch4]

print("Average Cross Entropy Loss for Train Set:", train_avCRE4)
print("Average Cross Entropy Loss for Test Set:", test_avCRE4)

print("Average Cross Entropy Loss per Epoch for Train Set:", train_avCRE_per_epoch4)
print("Average Cross Entropy Loss per Epoch for Test Set:", test_avCRE_per_epoch4)

print("Computing time per epoch: ", time_per_epoch4)


In [ ]:
# Plot the loss per epoch
plt.plot(train_losses_per_epoch4, label='Train')
plt.plot(test_losses_per_epoch4, label='Test')
plt.title('Cross-Entropy Loss per Epoch,  h_high=81')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Evaluate the model on the test set
test_loss4, test_accuracy4  = model4.evaluate(x_test, y_test, verbose=0)

print("Test Accuracy: {:.2f}%".format(test_accuracy4 * 100))


In [ ]:
test_pred_labels4 = np.argmax(test_preds4, axis=1)

# Compute the confusion matrix
conf_matrix4 = confusion_matrix(test_pred_labels4,y_test_labels)

# Normalize the confusion matrix
normalized_conf_matrix4 = conf_matrix4.astype('float') / conf_matrix4.sum(axis=1)[:, np.newaxis]

# Convert to percentage
percentage_conf_matrix4 = normalized_conf_matrix4 * 100

print("Confusion Matrix (Percentage):")
print("Confusion Matrix (Percentage):")
for row in percentage_conf_matrix4:
    formatted_row = ["{:.2f}".format(x) for x in row]
    print(" ".join(formatted_row))


# **Part 5**

In [ ]:
# Define the model, this time with seperate objects for each layer
input_layer = tf.keras.Input(shape=(x_train.shape[1],))
hidden_layer = tf.keras.layers.Dense(81, activation='sigmoid', kernel_initializer=RandomUniform)(input_layer)
output_layer = tf.keras.layers.Dense(y_train.shape[1], activation='softmax')(hidden_layer)

mlp_high = tf.keras.Model(inputs=input_layer, outputs=output_layer)

# Compile the model
mlp_high.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = mlp_high.fit(x_train, y_train, batch_size=42, epochs=150, verbose=0, validation_data=(x_test,y_test))


In [ ]:
# Create new model to study the hidden layer
hidden_layer_model = tf.keras.Model(inputs=mlp_high.input, outputs=mlp_high.layers[1].output)

# Compute Zn for each case in x_train
Zn_train = hidden_layer_model.predict(x_train)
Zn_test = hidden_layer_model.predict(x_test)


In [ ]:
latent_dim = 81
input_dim = 81
sparsity_target = 0.1
sparsity_weight = 1e-3

class Autoencoder(tf.keras.Model):
    def __init__(self, latent_dim, input_dim):
        super(Autoencoder, self).__init__()

        self.latent_dim = latent_dim   
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.Input(shape=(input_dim,)),
            tf.keras.layers.Dense(latent_dim, activation='sigmoid'),
        ])

        self.decoder = tf.keras.Sequential([
            tf.keras.layers.Dense(input_dim, activation='sigmoid'),
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

autoencoder = Autoencoder(latent_dim, input_dim)

# Define RMSE as a custom metric
def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))

# Define the KL divergence sparsity penalty
def kl_divergence(p, p_hat):
    return p * tf.math.log(p / p_hat) + (1 - p) * tf.math.log((1 - p) / (1 - p_hat))

# Define the sparse loss function
def sparse_loss(y_true, y_pred):
    mse_loss = tf.reduce_mean(tf.square(y_pred - y_true))
    hidden_layer = autoencoder.encoder(y_true)
    p_hat = tf.reduce_mean(hidden_layer, axis=0)
    sparsity_loss = tf.reduce_sum(kl_divergence(sparsity_target, p_hat))
    total_loss = mse_loss + sparsity_weight * sparsity_loss
    return total_loss

autoencoder.compile(optimizer='adam', loss=sparse_loss, metrics=[rmse])

history_aec = autoencoder.fit(Zn_train, Zn_train,
                              epochs=75,
                              shuffle=True,
                              validation_data=(Zn_test, Zn_test),
                              verbose=1)


In [ ]:
train_rmse = history_aec.history['rmse']
val_rmse = history_aec.history['val_rmse']
epochs = range(1, len(train_rmse) + 1)

plt.plot(epochs, train_rmse, 'b', label='Training RMSE')
plt.plot(epochs, val_rmse, 'r', label='Test RMSE')
plt.xlabel('Epochs')
plt.ylabel('RMSE')
plt.legend()
plt.title('RMSE vs. Epochs')

plt.show()


In [ ]:
# Copy and paste model3 from other file

# Define the model
model3 = tf.keras.Sequential([
    tf.keras.layers.Dense(81, activation='relu', input_shape=(x_train.shape[1],), kernel_initializer=RandomUniform),
    tf.keras.layers.Dense(y_train.shape[1], activation='softmax')
])

# Compile the model
model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history3 = model3.fit(x_train, y_train, batch_size=42, epochs=150, verbose=0, validation_data=(x_test,y_test))


In [ ]:
# Extract weights and biases from the first dense layer of model3 and the encoder
model3_dense1_weights, model3_dense1_biases = model3.layers[0].get_weights()

encoder_weights, encoder_biases = autoencoder.encoder.layers[0].get_weights()

# Verify that the weights and biases have the same dimensions
print("Model3 Weights Size: ", model3_dense1_weights.shape)
print("Model3 Biases Size: ", model3_dense1_biases.shape)
print("Encoder Weights Size: ", encoder_weights.shape)
print("Encoder Biases Size: ", encoder_biases.shape)


In [ ]:
Zn_combined = np.concatenate((Zn_train, Zn_test), axis=0)

# Define the modified MLP_12 model
MLP_12 = tf.keras.Sequential([
    model3.layers[0],
    autoencoder.encoder,
    tf.keras.layers.Dense(81, activation='linear')  # Change the output dimension to 82
])

# Compile the modified model
MLP_12.compile(optimizer='adam', loss='mse')

# Train the modified model
history_MLP_12 = MLP_12.fit(x_train, Zn_train,
                            batch_size=42,
                            epochs=150,
                            verbose=0,
                            validation_data=(x_test, Zn_test))

# Combine x_train and x_test
x_combined = np.concatenate((x_train, x_test), axis=0)
y_combined = np.concatenate((y_train, y_test), axis=0)

# Predict the output using the modified MLP_12 model
U_n = MLP_12.predict(x_combined)


In [ ]:
# Split the U_n dataset into training and test sets (80/20)
U_n_train, U_n_test, y_train_new, y_test_new = train_test_split(U_n, y_combined, test_size=0.2, random_state=42)
U_n.shape


In [ ]:
# Define the MLP3 model
MLP3 = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(U_n_train.shape[1],)),
    tf.keras.layers.Dense(40, activation='relu'),
    tf.keras.layers.Dense(y_train.shape[1], activation='softmax')
])

# Compile the MLP3 model
MLP3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the MLP3 model
history_MLP3 = MLP3.fit(U_n_train, y_train_new,
                        batch_size=42,
                        epochs=250,
                        verbose=0,
                        validation_data=(U_n_test, y_test_new))

train_preds =  MLP3.predict(U_n_train)
train_avCRE = log_loss(y_train_new, train_preds) / len(U_n_train)

test_preds = MLP3.predict(U_n_test)
test_avCRE = log_loss(y_test_new, test_preds) / len(U_n_test)

# Extract the training and validation losses for each epoch
train_losses_per_epoch = history_MLP3.history['loss']
test_losses_per_epoch = history_MLP3.history['val_loss']

# Calculate the average cross-entropy loss per epoch for train and test sets
train_avCRE_per_epoch = [loss / len(U_n_train) for loss in train_losses_per_epoch]
test_avCRE_per_epoch = [loss / len(U_n_test) for loss in test_losses_per_epoch]

print("Average Cross Entropy Loss for Train Set:", train_avCRE)
print("Average Cross Entropy Loss for Test Set:", test_avCRE)

print("Average Cross Entropy Loss per Epoch for Train Set:", train_avCRE_per_epoch)
print("Average Cross Entropy Loss per Epoch for Test Set:", test_avCRE_per_epoch)


In [ ]:
plt.plot(train_avCRE_per_epoch, label='Train')
plt.plot(test_avCRE_per_epoch, label='Test')
plt.title('Average Cross-Entropy Loss per Epoch, MLP3')
plt.xlabel('Epoch')
plt.ylabel('Average Loss')
plt.legend()
plt.show()

# Evaluate the MLP3 model on the U_n_test set
test_loss_MLP3, test_accuracy_MLP3 = MLP3.evaluate(U_n_test, y_test_new, verbose=0)

print("Test Accuracy: {:.2f}%".format(test_accuracy_MLP3 * 100))


In [ ]:
# Create and print the confusion matrix.
test_pred_labels = np.argmax(test_preds, axis=1)
y_test_array = y_test_new
y_test_labels = np.argmax(y_test_array, axis=1)

# Compute the confusion matrix
conf_matrix = confusion_matrix(test_pred_labels, y_test_labels)

# Normalize the confusion matrix
normalized_conf_matrix = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]

# Convert to percentage
percentage_conf_matrix = normalized_conf_matrix * 100

print("Confusion Matrix (Percentage):")
print("Confusion Matrix (Percentage):")
for row in percentage_conf_matrix:
    formatted_row = ["{:.2f}".format(x) for x in row]
    print(" ".join(formatted_row))

In [ ]:
MLP_long = tf.keras.Sequential([
    MLP_12.layers[0],  # First hidden layer from MLP_12
    MLP_12.layers[1],  # Second hidden layer (output layer of MLP_12)
    MLP3.layers[0],  # 
    MLP3.layers[1],  # Third hidden layer from MLP3
])


# Compile the MLP_long model
MLP_long.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Train the MLP_long model
history_MLP_long = MLP_long.fit(x_train, y_train,
                                batch_size=42,
                                epochs=50,
                                verbose=0,
                                validation_data=(x_test, y_test))

train_preds_long =  MLP_long.predict(x_train)
train_avCRE_long = log_loss(y_train, train_preds_long) / len(x_train)

test_preds_long = MLP_long.predict(x_test)
test_avCRE_long = log_loss(y_test, test_preds_long) / len(x_test)

# Extract the training and validation losses for each epoch
train_losses_per_epoch_long = history_MLP_long.history['loss']
test_losses_per_epoch_long = history_MLP_long.history['val_loss']

# Calculate the average cross-entropy loss per epoch for train and test sets
train_avCRE_per_epoch_long = [loss / len(x_train) for loss in train_losses_per_epoch_long]
test_avCRE_per_epoch_long = [loss / len(x_test) for loss in test_losses_per_epoch_long]

print("Average Cross Entropy Loss for Train Set:", train_avCRE_long)
print("Average Cross Entropy Loss for Test Set:", test_avCRE_long)

print("Average Cross Entropy Loss per Epoch for Train Set:", train_avCRE_per_epoch_long)
print("Average Cross Entropy Loss per Epoch for Test Set:", test_avCRE_per_epoch_long)


In [ ]:
plt.plot(train_avCRE_per_epoch_long, label='Train')
plt.plot(test_avCRE_per_epoch_long, label='Test')
plt.title('Average Cross-Entropy Loss per Epoch, MLP_long')
plt.xlabel('Epoch')
plt.ylabel('Average Loss')
plt.legend()
plt.show()

# Evaluate the MLP3 model on the U_n_test set
test_loss_long, test_accuracy_long = MLP_long.evaluate(x_test, y_test, verbose=0)

print("Test Accuracy: {:.2f}%".format(test_accuracy_long * 100))
